<a href="https://colab.research.google.com/github/wonjonghy/Project-Section-1/blob/main/Part1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 확인하세요!

### 제출 전에 모든 코드가 순차적으로 잘 작동하는지 확인하세요. 
- **런타임 다시 시작**한 다음(메뉴바에서 런타임 $\rightarrow$ 런타임 다시 시작 선택) **모든 셀을 실행**해 보세요(메뉴바에서 런타임 $\rightarrow$ 모두 실행 선택).

### 지정 셀의 지정 영역에서만 코드를 작성했는지 확인하세요.
- `YOUR CODE HERE`이 적혀 있는 셀에 대한 코드만 채점에 반영됩니다.
- `YOUR CODE HERE` ~ `YOUR CODE ENDS HERE` 사이에 본인의 코드를 작성하세요.
- 지정 셀 이외에 본인이 코드 실행 결과를 확인하기 위한 용도로 셀을 추가해도 됩니다.
    - **제출 시 지정 셀에서는 정답 이외에 결과 확인을 위한 코드 등은 제거하고 제출해 주세요!**
    - **기존 셀을 삭제하지 마세요! 채점에 불이익이 있을 수 있습니다.**

### 제출 방법
- 시간 내에 문제를 풀고, 본인의 github repo에 push 하세요.
    - **디렉토리 구조가 변하면 안 됩니다.** `codestates/ds-sc2xx`와 디렉토리 구조가 동일해야 합니다. 답안 파일의 이름을 변경하지 않았는지, 기존에 없던 폴더가 추가되지 않았는지 등을 확인하세요.
    - **코드스테이츠 가입 시 연동한 github 계정을 사용해야 합니다.**
    - **branch를 변경하여 작업하지 마세요.**
        - `master` branch에의 제출 사항에 대해서만 평가합니다.
        - 본인의 github repo `master` branch에 제출이 잘 되어 있는지 확인하세요.

---

# SC22x
 
### 코비 브라이언트의 슛 성공 여부 예측해보기

이번 스프린트 챌린지에서는 故 코비 브라이언트 선수의 NBA 필드 골 시도들이 담긴 데이터셋을 다룹니다 (1996년 11월 3일부터 2016년 4월 13일까지의 정규 시즌과 플레이오프 시즌 기록).

여러분은 각 슛의 성공 여부를 슈팅의 대한 정보, 게임에 대한 정보를 통해서 예측해야 합니다. 이것은 상당히 어려운 예측입니다. 해당 데이터셋은 [NBA API]((https://github.com/swar/nba_api) 파이썬 라이브러리를 통해 수집 됐습니다. 

In [1]:
import sys
import warnings

!pip3 install xgboost

if "google.colab" in sys.modules:
    ## Colab에서 실행하는 경우 필요한 라이브러리를 미리 설치합니다.
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 373 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 245 kB 4.9 MB/s 
     |████████████████████████████████| 4.7 MB 37.2 MB/s 
     |████████████████████████████████| 690 kB 46.3 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 
     |████████████████████████████████| 102 kB 40.7 MB/s 
     |████████████████████████████████| 355 kB 57.0 MB/s 
     |████████████████████████████████| 296 kB 49.6 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=1969c6c420809433fcc1635509cfbc5aafe692774e88d73a22c6cecaa20a429f
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d20

In [174]:
# 데이터셋을 불러옵니다
import pandas as pd

url = "https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/nba/nba_kobe_bryant.csv"
df = pd.read_csv(url)

# 데이터셋이 잘 불러졌는지 확인합니다
assert df.shape == (30697, 19)

In [175]:
df.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type
0,29600027,102,Kobe Bryant,1,0,42,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1996-11-03,LAL,MIN,Regular Season
1,29600031,127,Kobe Bryant,2,10,8,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,1996-11-05,NYK,LAL,Regular Season
2,29600044,124,Kobe Bryant,2,8,37,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,1996-11-06,CHH,LAL,Regular Season
3,29600044,144,Kobe Bryant,2,6,34,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,1996-11-06,CHH,LAL,Regular Season
4,29600044,151,Kobe Bryant,2,5,27,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,1996-11-06,CHH,LAL,Regular Season


In [ ]:
# 살펴보기
df.isnull().sum()

# Part 1 - Data Preprocessing

## 1.1 분류를 위한 베이스라인 정확도를 계산하는 `calculate_baseline_score` 함수를 작성하세요.
- 예측해야 하는 target은 `shot_made_flag` 입니다. 
- 전체 데이터를 써서 baseline **accuracy score**를 계산하세요.
- 최빈 클래스를 사용하는 baseline을 가정하세요.

In [6]:
df.shot_made_flag.unique()

array([0, 1])

In [7]:
df.shot_made_flag.sum()

13731

In [8]:
def calculate_baseline_score(df):
    accuracy_score = 0

    # YOUR CODE HERE
    accuracy_score = df.shot_made_flag.sum()/len(df) * 100
    
    # YOUR CODE ENDS HERE

    return accuracy_score

In [9]:
baseline_accuracy_score = calculate_baseline_score(df)

In [10]:
print("baseline accuracy score:", baseline_accuracy_score)

baseline accuracy score: 44.73075544841515


## 1.2 데이터셋을 아래의 기준을 사용해서 train/test 셋으로 나누는 `split_data` 함수를 작성하세요.

- `test set`: **2014-15** ~ **2015-16** 시즌
- NBA 시즌은 **10월**에 시작해서 **6월**에 종료합니다. 

In [11]:
df.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type
0,29600027,102,Kobe Bryant,1,0,42,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1996-11-03,LAL,MIN,Regular Season
1,29600031,127,Kobe Bryant,2,10,8,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,1996-11-05,NYK,LAL,Regular Season
2,29600044,124,Kobe Bryant,2,8,37,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,1996-11-06,CHH,LAL,Regular Season
3,29600044,144,Kobe Bryant,2,6,34,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,1996-11-06,CHH,LAL,Regular Season
4,29600044,151,Kobe Bryant,2,5,27,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,1996-11-06,CHH,LAL,Regular Season


In [63]:
pd.set_option('display.max_rows', None)

In [38]:
def split_data(df):
    train = None
    test = None

    # YOUR CODE HERE
    import datetime as dt
    df.game_date = pd.to_datetime(df.game_date)
    test = df[df.game_date.dt.year >= 2014]
    train = df[df.game_date.dt.year < 2014]
    
    # YOUR CODE ENDS HERE

    return train, test

In [39]:
train, test = split_data(df.copy())

In [76]:
test.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type
28871,21400003,76,Kobe Bryant,1,6,13,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,20,-54,201,0,2014-10-28,LAL,HOU,Regular Season
28872,21400003,94,Kobe Bryant,1,4,45,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,-16,63,1,2014-10-28,LAL,HOU,Regular Season
28873,21400003,413,Kobe Bryant,3,5,52,Turnaround Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),16-24 ft.,18,166,78,0,2014-10-28,LAL,HOU,Regular Season
28874,21400003,520,Kobe Bryant,4,9,36,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,27,-138,241,0,2014-10-28,LAL,HOU,Regular Season
28875,21400003,553,Kobe Bryant,4,7,16,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-179,88,0,2014-10-28,LAL,HOU,Regular Season


In [44]:
train.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type
0,29600027,102,Kobe Bryant,1,0,42,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1996-11-03,LAL,MIN,Regular Season
1,29600031,127,Kobe Bryant,2,10,8,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,1996-11-05,NYK,LAL,Regular Season
2,29600044,124,Kobe Bryant,2,8,37,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,1996-11-06,CHH,LAL,Regular Season
3,29600044,144,Kobe Bryant,2,6,34,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,1996-11-06,CHH,LAL,Regular Season
4,29600044,151,Kobe Bryant,2,5,27,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,1996-11-06,CHH,LAL,Regular Season


In [115]:
train.game_id.unique()

array([29600027, 29600031, 29600044, ..., 21300342, 21300357, 21300366])

In [161]:
mps = pd.Series([])

In [160]:
df.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,Opponent
0,29600027,102,Kobe Bryant,1,0,42,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1996-11-03,LAL,MIN,Regular Season,LAL
1,29600031,127,Kobe Bryant,2,10,8,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,1996-11-05,NYK,LAL,Regular Season,NYK
2,29600044,124,Kobe Bryant,2,8,37,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,1996-11-06,CHH,LAL,Regular Season,CHH
3,29600044,144,Kobe Bryant,2,6,34,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,1996-11-06,CHH,LAL,Regular Season,CHH
4,29600044,151,Kobe Bryant,2,5,27,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,1996-11-06,CHH,LAL,Regular Season,CHH


In [ ]:
mps = [-1]*len(df)
mps

In [ ]:
mps = [-1]*len(df)
mps[0] = 0
mps

In [190]:
df.shot_made_flag.dtype

dtype('int64')

In [192]:
df.shot_made_flag[4]

1

In [202]:
mps = [-1]*len(df)
mps[0] = 'None'

for i in range(0, len(df)-1):
  if df.shot_made_flag[i] == 0:
      mps[i+1] = 0
  else:
      mps[i+1] = 1



In [ ]:
mps

In [185]:
df['mps'] = mps

In [186]:
df.head()

,game_id,game_event_id,player_name,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,mps
0,29600027,102,Kobe Bryant,1,0,42,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-140,116,0,1996-11-03,LAL,MIN,Regular Season,None
1,29600031,127,Kobe Bryant,2,10,8,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,16,-131,97,0,1996-11-05,NYK,LAL,Regular Season,None
2,29600044,124,Kobe Bryant,2,8,37,Jump Shot,3PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,23,-142,181,1,1996-11-06,CHH,LAL,Regular Season,None
3,29600044,144,Kobe Bryant,2,6,34,Jump Shot,3PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,0,1996-11-06,CHH,LAL,Regular Season,None
4,29600044,151,Kobe Bryant,2,5,27,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,13,-10,138,1,1996-11-06,CHH,LAL,Regular Season,None


In [ ]:
df[['htm', 'vtm']]

## 1.3 새로운 feature를 만드는 `feature_engineering` 함수를 작성하세요.

- 새로운 이름을 갖는 최소 2개 이상의 새로운 feature를 만드세요. 아래의 리스트를 사용하셔도 되고 스스로의 아이디어로 만들어 주셔도 괜찮습니다.
    - **Homecourt Advantage**: 홈 팀(`htm`)이 Los Angeles Lakers (`LAL`)인지 여부
    - **Opponent**: LA Lakers를 상대하는 팀을 나타내는 feature
    - **Seconds remaining in the period**: 피리어드에 남은 총 시간 (분과 초를 합칩니다)
    - **Seconds remaining in the game**: 게임에 남은 총 시간 (피리어드와 초를 합칩니다). 농구에서 게임은 총 4개의 피리어드로 이루어져 있습니다 (각 12분)
    - **Made previous shot**: 코비 브라이언트의 예전 슛이 성공했는지 여부
- 학습에 사용하지 않는 특성은 drop하세요.
    

In [211]:
def feature_engineering(df):
    df_tmp = df.copy()

    # YOUR CODE HERE
    df_tmp['Homecourt Advantage'] = df_tmp.htm.map(lambda x: 1 if 'LAL' in x else 0)
    df_tmp['Seconds remaining in the period'] = df_tmp['minutes_remaining']*60 + df_tmp['seconds_remaining']
    df_tmp['Seconds remaining in the game'] = df_tmp['period'].map(lambda x: (4 - x)*12*60) + df_tmp['Seconds remaining in the period']

    df_tmp.drop(['player_name', 'game_id', 'game_event_id'], axis = 1, inplace = True)
    
    list = []
    df_tmp.reset_index(drop = True, inplace = True)
    for i in range(len(df_tmp)):
      if df_tmp.htm[i] != 'LAL':
        list.append(df_tmp.htm[i])
      elif df_tmp.vtm[i] != 'LAL':
        list.append(df_tmp.vtm[i])
    df_tmp['Opponent'] = list

    mps = [-1]*len(df_tmp)
    mps[0] = 'None'

    for i in range(0, len(df_tmp)-1):
      if df_tmp.shot_made_flag[i] == 0:
        mps[i+1] = 0
    else:
      mps[i+1] = 1
    df_tmp['Made previous shot'] = mps
    # YOUR CODE ENDS HERE

    return df_tmp

In [207]:
    mps = [-1]*len(df)
    mps[0] = 'None'

In [209]:
len(test)

1826

In [210]:
test.reset_index(drop = True, inplace = True)
for i in range(0, len(test)-1):
  if test.shot_made_flag[i] == 0:
    mps[i+1] = 0
  else:
    mps[i+1] = 1
test['Made previous shot'] = mps

ValueError: ignored

In [212]:
train_processed = feature_engineering(train)
test_processed = feature_engineering(test)

In [ ]:
train_processed.head()

In [111]:
test_processed.head()

,period,minutes_remaining,seconds_remaining,action_type,shot_type,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,loc_x,loc_y,shot_made_flag,game_date,htm,vtm,season_type,Homecourt Advantage,Seconds remaining in the period,Seconds remaining in the game,Opponent
0,1,6,13,Jump Shot,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,20,-54,201,0,2014-10-28,LAL,HOU,Regular Season,1,373,2533,HOU
1,1,4,45,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,-16,63,1,2014-10-28,LAL,HOU,Regular Season,1,285,2445,HOU
2,3,5,52,Turnaround Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),16-24 ft.,18,166,78,0,2014-10-28,LAL,HOU,Regular Season,1,352,1072,HOU
3,4,9,36,Jump Shot,3PT Field Goal,Above the Break 3,Left Side Center(LC),24+ ft.,27,-138,241,0,2014-10-28,LAL,HOU,Regular Season,1,576,576,HOU
4,4,7,16,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-179,88,0,2014-10-28,LAL,HOU,Regular Season,1,436,436,HOU


## 1.4 train / test 셋을 features(X)와 target(y)으로 각각 나누는 `divide_data` 함수를 작성하세요.
- `pd.DataFrame` 형태의 `X`와 `pd.Series` 형태의 `y`를 반환하세요.

In [90]:
df.drop('shot_made_flag', axis = 1).columns

Index(['game_id', 'game_event_id', 'player_name', 'period',
       'minutes_remaining', 'seconds_remaining', 'action_type', 'shot_type',
       'shot_zone_basic', 'shot_zone_area', 'shot_zone_range', 'shot_distance',
       'loc_x', 'loc_y', 'game_date', 'htm', 'vtm', 'season_type', 'Opponent'],
      dtype='object')

In [91]:
def divide_data(df):
    X = None
    y = None

    # YOUR CODE HERE
    features = df.drop('shot_made_flag', axis = 1).columns
    target = 'shot_made_flag'
    X = df[features]
    y = df[target]
    
    # YOUR CODE ENDS HERE

    return X, y

In [92]:
X_train, y_train = divide_data(train_processed)
X_test, y_test = divide_data(test_processed)

In [93]:
X_train.shape, X_test.shape

((28871, 19), (1826, 19))

# Part 2 - Modeling & Evaluation

## 2.1 RandomForest 또는 XGBoost 모델 학습 파이프라인을 생성하고 학습하는 `fit_trees` 함수를 작성하세요. 
- Scikit-learn의 `pipeline`을 활용하여 범주형 feature의 encoder, imputer, 모델을 포함하는 하나의 학습 파이프라인을 구축하세요.
- 구축한 파이프라인에 대해 `RandomizedSearchCV`를 사용해 CV를 진행하세요.
    - 적어도 두 개 이상의 하이퍼파라미터에 대해 탐색을 진행하세요.
    - F1 score을 최대화하는 하이퍼파라미터 조합을 선택하세요.
- RandomForestClassifier, RandomizedSearchCV 등 랜덤성이 포함되는 방법론에 대해 모두 `random_state=42`로 하세요.
- cv는 5 이하, n_iter은 10회 이하로 하세요.

In [113]:
import sklearn
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from category_encoders import OrdinalEncoder

### IMPORT ANY
# YOUR CODE HERE
from scipy.stats import randint, uniform

# YOUR CODE ENDS HERE


def fit_trees(X_train, y_train):
    pipe = None    # pipe = make_pipeline(...)
    clf = None     # clf = RandomizedSearchCV(pipe, ...)

    # YOUR CODE HERE
    X_train.game_date = X_train.game_date.astype('object')
    pipe = make_pipeline(
        OrdinalEncoder(),
        SimpleImputer(),
        RandomForestClassifier(),
        )
    
    dists = {
        'simpleimputer__strategy': ['mean', 'median'], 
        'randomforestclassifier__n_estimators': randint(50, 500), 
        'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
        'randomforestclassifier__max_features': uniform(0, 1)
        }
    clf = RandomizedSearchCV(
        pipe, 
        param_distributions=dists, 
        n_iter=10, 
        cv=3,
        scoring='f1',
        verbose=1,
        n_jobs=-1
        )

    clf.fit(X_train, y_train)
    # YOUR CODE ENDS HERE

    return clf

In [114]:
clf = fit_trees(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [116]:
clf.best_params_

{'randomforestclassifier__max_depth': 10,
 'randomforestclassifier__max_features': 0.18283236278301707,
 'randomforestclassifier__n_estimators': 249,
 'simpleimputer__strategy': 'median'}

In [117]:
print("Best CV F1", clf.best_score_)
print("Best CV params", clf.best_params_)

Best CV F1 0.5803666408648067
Best CV params {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__max_features': 0.18283236278301707, 'randomforestclassifier__n_estimators': 249, 'simpleimputer__strategy': 'median'}


## 2.2 학습된 모델로부터 feature importance를 계산하는 `get_feature_importances` 함수를 작성하세요. 
- feature_importances 변수에 feature_importances_ 값을 할당하면 됩니다.

In [134]:
import numpy as np


def get_feature_importances(pipe, feature_names):
    feature_importances = np.zeros(len(feature_names))

    # YOUR CODE HERE
    random_fc = pipe.named_steps['randomforestclassifier']
    feature_importances = random_fc.feature_importances_
    
    # YOUR CODE ENDS HERE

    return sorted(
        list(zip(feature_names, feature_importances)), key=lambda x: x[1], reverse=True
    )

In [141]:
pipe = clf.best_estimator_

In [142]:
    random_fc = pipe.named_steps['randomforestclassifier']
    feature_importances = random_fc.feature_importances_

In [143]:
feature_importances

array([0.01256356, 0.01951401, 0.03606615, 0.40867742, 0.01262562,
       0.02487467, 0.01756292, 0.01858752, 0.09037307, 0.04935723,
       0.06571221, 0.05302178, 0.02835113, 0.022737  , 0.00475656,
       0.00809293, 0.04853518, 0.04744639, 0.03114464])

In [147]:
feature_names = X_train.columns

In [148]:
list(zip(feature_names, feature_importances)), key=lambda x: x[1], reverse=True

SyntaxError: ignored

In [139]:
np.zeros(len(X_train.columns))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [ ]:
clf.best_estimator_

In [130]:
X_train.columns.to_list()

['period',
 'minutes_remaining',
 'seconds_remaining',
 'action_type',
 'shot_type',
 'shot_zone_basic',
 'shot_zone_area',
 'shot_zone_range',
 'shot_distance',
 'loc_x',
 'loc_y',
 'game_date',
 'htm',
 'vtm',
 'season_type',
 'Homecourt Advantage',
 'Seconds remaining in the period',
 'Seconds remaining in the game',
 'Opponent']

In [138]:
X_train.columns

Index(['period', 'minutes_remaining', 'seconds_remaining', 'action_type',
       'shot_type', 'shot_zone_basic', 'shot_zone_area', 'shot_zone_range',
       'shot_distance', 'loc_x', 'loc_y', 'game_date', 'htm', 'vtm',
       'season_type', 'Homecourt Advantage', 'Seconds remaining in the period',
       'Seconds remaining in the game', 'Opponent'],
      dtype='object')

In [150]:
feature_importances = get_feature_importances(
    clf.best_estimator_, list(X_train.columns)
)

TypeError: ignored

In [135]:
feature_importances = get_feature_importances(
    clf.best_estimator_, list(X_train.columns)
)

TypeError: ignored

In [ ]:
for name, importance in feature_importances:
    print(f"{name}: {importance:.4f}")

## 2.3 본인이 만든 모델의 Test 셋에서의 성능을 확인해 보세요.
- ### test_accuracy > 0.6, test_f1 > 0.5를 만족하면 기본 성능점수를 얻습니다.
- ###  (도전과제) test_accuracy > 0.68, test_f1 > 0.54을 만족하면 성능점수 만점을 얻습니다.
  - 도전과제까지 수행해야 스챌 3점을 얻습니다. 
  - 1.3으로 돌아가 다양한 특성 조합을 시도해 보세요. 
    - 그리고 2.2로 돌아와 피쳐 중요도를 계산해 보세요.
    - 어떤 피쳐의 중요도가 가장 높나요?
    - 중요도가 낮은 피쳐라도 있는 게 좋을까요?
  - 2.1로 돌아가 SearchCV에서 다양한 하이퍼파라미터 옵션 조합을 선택해 보세요.
    - 본인이 선택한 모델에서, 각 하이퍼파라미터가 어떤 의미를 가지나요?
    - SearchCV가 더 다양한 하이퍼파라미터 조합을 시도하도록 하기 위해 SearchCV의 어떤 옵션을 조정해야 할까요?

In [151]:
def calculate_score(pipe, X_test, y_test):
    test_accuracy = pipe.score(X_test, y_test)
    test_f1 = f1_score(y_test, pipe.predict(X_test))
    return test_accuracy, test_f1

In [152]:
test_accuracy, test_f1 = calculate_score(clf.best_estimator_, X_test, y_test)

In [153]:
print("test accuracy: ", test_accuracy)
print("test F1:", test_f1)

test accuracy:  0.6768893756845564
test F1: 0.5376175548589341


In [154]:
print("test accuracy > 0.68?: ", test_accuracy > 0.68)
print("test F1 > 0.54?:", test_f1 > 0.54)


test accuracy > 0.68?:  False
test F1 > 0.54?: False


---

# 수고하셨습니다!